In [11]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from pogdb.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob

In [12]:
# Import api/access_token keys from credentials.py
import credentials
auth  = tweepy.OAuthHandler(credentials.API_KEY, \
                            credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN,  \
                      credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [13]:
#The fuctions are used to clean the tweets
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [14]:
#This connnects to the database, checks if a table {"Tablename"} exits, if not creates one and closes the connection
import os
import psycopg2

dbconn = psycopg2.connect("host=ec2-54-81-37-115.compute-1.amazonaws.com dbname=d58g5m66umb113 user=eigxqdhsrlaffv password=b1495696080d3ad38656b3e2973e25cd8890a1570a7639abdf756cbbd793c8f1")
if dbconn:
    print("Connected")
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = dbconn.cursor()
    """mycursor.execute(
    SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = '{0}'.format(settings.TABLE_NAME))
    print("table exists")
    if mycursor.fetchone()[0] != 1:"""
    mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
    print("Table does not exist so it has been created")
    dbconn.commit()
    mycursor.close()
else:
    print('Not connected')

Connected


DuplicateTable: relation "nairobitweets" already exists


In [ ]:
#creating a listener to watch for our data
#has two functions
#on_status it to check for tweets
#on_error is to stop tweet checking incase a limit is reached
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            print(longitude)
            print(latitude)
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if dbconn:
            mycursor = dbconn.cursor()
            sql = "INSERT INTO {} (id_str,created_at,text,polarity,\
                subjectivity, user_created_at, user_location,\
                user_description, user_followers_count, longitude,\
                latitude, retweet_count, favorite_count) VALUES \
                (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity,\
                 user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            dbconn.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [ ]:
#This calls the class myStreamListener thereby witing into the database
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], locations = [-1.1597918307560573, 37.106463945682584, -1.3891756881977984, 36.665974517587834], """track = settings.TRACK_WORDS""")
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
dbconn.close()

In [30]:
#This is used to read data from the database
import psycopg2
try:
    connection = psycopg2.connect(user="eigxqdhsrlaffv",
                                  password="b1495696080d3ad38656b3e2973e25cd8890a1570a7639abdf756cbbd793c8f1",
                                  host="ec2-54-81-37-115.compute-1.amazonaws.com",
                                  database="d58g5m66umb113")
    cursor = connection.cursor()
    postgreSQL_select_Query = "select * from tweets"

    cursor.execute(postgreSQL_select_Query)
    print("Selecting rows from tweets table using cursor.fetchall")
    tweet_records = cursor.fetchall() 
   
    print("Print each row and it's columns values")
    for row in tweet_records:
        print("Id = ", row[0], )
        print("created_at = ", row[1], )
        print("text  = ", row[2], )
        print("polarity = ", row[3], )
        print("subjectivity = ", row[4], "\n")

except (Exception, psycopg2.Error) as error :
    print ("Error while fetching data from PostgreSQL", error)

finally:
    #closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed").close()

Selecting rows from tweets table using cursor.fetchall
Print each row and it's columns values
Id =  1260933702965587970
created_at =  2020-05-14 14:03:16
text  =  RT @Only_rock_radio: Great music now: Wonder by TolbertToz @tolberttoz follow @tiorr_2 to all the spins (click the link) https://t.co/pyHWG
polarity =  1
subjectivity =  1 

Id =  1260933702839721985
created_at =  2020-05-14 14:03:16
text  =  RT @dearchengxiao: [200514]

 #chengxiao oasis and weibo update
  sangsangs red wedding dress  memories in xinjiang~ 

 # # #
polarity =  0
subjectivity =  1 

Id =  1260933702894280716
created_at =  2020-05-14 14:03:16
text  =  RT @Sidfan222: Hey #SidHearts 
I want to make a thread of #SidharthShukla sir as a rose....
@SidShukla_1 @OfficialSidFC @TeamSidharthFC @Ne
polarity =  1
subjectivity =  1 

Id =  1260933702957150219
created_at =  2020-05-14 14:03:16
text  =  RT @memefess: Yeet! https://t.co/R0jDAXBy4e
polarity =  0
subjectivity =  0 

Id =  1260933702936190980
created_at =  2020

polarity =  0
subjectivity =  0 

Id =  1260933720476758016
created_at =  2020-05-14 14:03:20
text  =  RT @dailydoseofMG: They came for us today. 
-Matching outfits
-Pajamas
-Chicken suits
-Chopper 
-Singing 
-indirect kissing 
-escaping upst
polarity =  -1
subjectivity =  1 

Id =  1260933720497946624
created_at =  2020-05-14 14:03:20
text  =  RT @deartori_: SAME SIS lmaoooo
polarity =  0
subjectivity =  0 

Id =  1260933720455786498
created_at =  2020-05-14 14:03:20
text  =  RT @eyezonyou__: M:have u read the script(TTTS ss2) yet?
G:no
M:not yet?
G:just a bit, skimming
M:when it's workshop, we gotta practice
G:c
polarity =  0
subjectivity =  0 

Id =  1260933719726030856
created_at =  2020-05-14 14:03:20
text  =  RT @JDAMgmIndore: 20 Days yet no Justice 
Why was the enquiry cancelled?
Why r they nt letting d truth be spoken?
 #sayNoTooppression #saty
polarity =  0
subjectivity =  0 

Id =  1260933720493580288
created_at =  2020-05-14 14:03:20
text  =  RT @jeannaesugar: i need https:/

AttributeError: 'NoneType' object has no attribute 'close'